## Function Calling

In [1]:
from openai import AzureOpenAI
client = AzureOpenAI()

In [2]:
response = client.chat.completions.create(model='myllm',
                                          messages=[{"role":"user","content":"hi"}])
print(response.choices[0].to_json(indent=2))

{
  "finish_reason": "stop",
  "index": 0,
  "logprobs": null,
  "message": {
    "content": "Hello! How can I assist you today?",
    "refusal": null,
    "role": "assistant",
    "annotations": []
  },
  "content_filter_results": {
    "hate": {
      "filtered": false,
      "severity": "safe"
    },
    "self_harm": {
      "filtered": false,
      "severity": "safe"
    },
    "sexual": {
      "filtered": false,
      "severity": "safe"
    },
    "violence": {
      "filtered": false,
      "severity": "safe"
    }
  }
}


In [3]:
import os
import json
import requests

In [10]:
def get_current_weather(city:str)->dict:
    """ this function can be used to fetch weather information for a given city"""
    api_key=os.environ['OPENWEATHERMAP_API_KEY']

    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}"
    response = requests.get(url)
    response = response.content.decode() # bytes to string
    response = json.loads(response) # string to dict
    output = {"city":city,
              "weather":response['weather'][0]['description'],
              "temperature":response['main']['temp'],
              "unit":"kelvin"}
    return output


In [11]:
get_current_weather("mumbai")

{'city': 'mumbai', 'weather': 'haze', 'temperature': 302.14, 'unit': 'kelvin'}

#### There 3 important components to be provided to LLM as metadata about functions
1. function name: should be descriptive, it should define what function does, this does not have to be same as the actual name of function
2. function description: should be descriptive, if possible include example of calling, - core basis on which LLM relies on selecting the right function
3. Arguments: list of params the function supports with example,description of each parameter

In [12]:
tools = [{"type":"function",
          "function":{
              "name":"get_current_weather",
              "description":"this function can be used to current weather information for a given city.",
              "parameters":{
                  "type":"object",
                  "properties":{"city":{"type":"string","description":"name of city/location. e.g. mumbai, new york"},
                                },
                                "required":['city'],
              }
          }}]

In [21]:
tool_names = {"get_current_weather":get_current_weather} # name of function in metadata : name of object

def generate_response(prompt):
    messages = [{"role":"system","content":"you are a helpful assistant, who answers questions in concise way"},
                {"role":"user","content":prompt},]
    
    first_response = client.chat.completions.create(model="myllm",messages=messages,
                                                    temperature=0.1,tools=tools,
                                                    tool_choice='auto')
    
    first_response = first_response.choices[0].message
    if first_response.tool_calls:
        # do something
        print(first_response)
        messages.append(first_response)

        # iterate through all the functions, execute and log responses in messages
        for tool in first_response.tool_calls:
            tool_name = tool.function.name
            tool_args = json.loads(tool.function.arguments) # json.loads converts str to dict
            fun_toexecute = tool_names[tool_name]
            tool_output = fun_toexecute(**tool_args)
            messages.append({"tool_call_id":tool.id,
                             "role":"tool","name":tool_name,"content":json.dumps(tool_output)})
        second_response = client.chat.completions.create(model="myllm",messages=messages,temperature=0.1)
        return second_response.choices[0].message.content
        
    else:
        return first_response.content

In [22]:
op = generate_response("what is quantum computing")
print(op)

Quantum computing is a type of computation that utilizes the principles of quantum mechanics to process information. Unlike classical computers, which use bits as the smallest unit of data (0s and 1s), quantum computers use quantum bits, or qubits. Qubits can exist in multiple states simultaneously due to superposition, allowing quantum computers to perform complex calculations at much higher speeds than classical computers for certain tasks.

Key concepts in quantum computing include:

1. **Superposition**: Qubits can represent both 0 and 1 at the same time, enabling parallel processing.
2. **Entanglement**: Qubits can be interconnected in such a way that the state of one qubit can depend on the state of another, no matter the distance between them.
3. **Quantum gates**: Operations that manipulate qubits, similar to logic gates in classical computing.

Quantum computing has the potential to revolutionize fields such as cryptography, optimization, and drug discovery by solving problems

In [23]:
op = generate_response("what is the current weather in new york today.")
print(op)

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_rbCUqjD3X8IJDFU0FsYfyYXZ', function=Function(arguments='{"city":"new york"}', name='get_current_weather'), type='function')])
The current weather in New York today is hazy, with a temperature of approximately 29.1°C (84.4°F).


In [ ]:
###